In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [1]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [4]:
import gzip
import simplejson as json

In [5]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [21]:
scripts[:2]

,act_cost,bnf_code,bnf_name,items,nic,practice,quantity
0,5.56,0101010G0AAABAB,Co-Magaldrox_Susp 195mg/220mg/5ml S/F,2,5.98,N81013,1000
1,1.82,0101021B0AAAHAH,Alginate_Raft-Forming Oral Susp S/F,1,1.95,N81013,500


A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [6]:
practices[:2]

NameError: name 'json_normalize' is not defined

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [5]:
def describe(key):
    total = 0
    avg = 0
    s = 0
    q25 = 0
    med = 0
    q75 = 0
    total = scripts[key].sum()
    avg = scripts[key].mean()
    s = scripts[key].std()
    q25 = scripts[key].quantile(0.25)
    med = scripts[key].quantile(0.5)
    q75 = scripts[key].quantile(0.75)
    return (total, avg, s, q25, med, q75)

In [6]:
from pandas.io.json import json_normalize
scripts = json_normalize(scripts)
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [11]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [9]:
bnf_names = scripts.groupby(['bnf_name'])
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [10]:
groups = {name[0]: name[1]['items'].sum() for name in bnf_names}

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [19]:
key_max = max(groups.keys(), key=(lambda k: groups[k]))
max_item = [(key_max, groups[key_max])]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [20]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [95]:
def group_by_field(data, fields):
    groups = {}
    for field in fields:
       # creates dict for each field with their unique element as their key
        groups[field] = {name: [] for name in set(dt[field] for dt in data)}
      # iterates rows in data and stores items corresponding to fieldname in field dict of groups
        for row in data:
            groups[field][row[field]].append(row['items'])
    return groups

In [96]:
groups = group_by_field(scripts, ('bnf_name','post_code'))
print(groups)
test_max_item = ...

assert test_max_item == max_item

{'bnf_name': {'Maxitrol_Eye Dps': [2, 1, 2, 1, 2, 5, 6, 4, 1, 2, 2, 2, 4, 1, 2, 1, 2, 2, 2, 1, 1, 5, 2, 2, 1, 2, 3, 4, 2, 1, 3, 2, 3, 7, 1, 4, 2, 3, 1, 1, 4, 1, 2, 1, 6, 3, 9, 2, 1, 1, 1, 4, 6, 1, 18, 8, 4, 13, 4, 1, 3, 3, 7, 5, 6, 3, 3, 5, 1, 1, 7, 5, 6, 2, 1, 1, 1, 1, 1, 1, 5, 3, 2, 2, 2, 1, 1, 1, 3, 2, 2, 3, 4, 2, 2, 1, 2, 1, 1, 1, 1, 4, 3, 1, 2, 1, 1, 3, 2, 4, 1, 2, 2, 2, 2, 1, 2, 5, 1, 1, 5, 7, 1, 2, 1, 2, 1, 2, 1, 1, 3, 3, 1, 2, 4, 1, 1, 2, 1, 3, 4, 3, 1, 1, 1, 7, 4, 1, 1, 3], 'Hylo-Forte Sod Hyaluronate Eye Dps 0.2%': [7, 9, 11, 13, 14, 8, 9, 19, 34, 39, 31, 3, 16, 7, 16, 2, 1, 4, 1, 1, 1, 4, 9, 10, 12, 11, 11, 3, 4, 6, 8, 4, 1, 2, 9, 9, 2, 2, 4, 7, 1, 21, 3, 3, 14, 29, 3, 8, 14, 6, 14, 7, 5, 11, 18, 4, 1, 2, 2, 17, 7, 7, 1, 6, 4, 1, 7, 16, 3, 2, 3, 3, 3, 1, 5, 10, 7, 18, 9, 12, 14, 1, 2, 4, 2, 16, 2, 7, 1, 2, 3, 3, 1, 2, 2, 52, 30, 47, 35, 8, 12, 7, 4, 6, 4, 37, 30, 38, 30, 51, 52, 80, 32, 17, 27, 9, 7, 13, 7, 23, 2, 4, 3, 2, 2, 4, 6, 3, 1, 7, 9, 5, 1, 3, 9, 2, 4, 8, 3, 7, 4, 4

NameError: name 'max_item' is not defined

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [8]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = practice['post_code'] if practice['post_code']<practice_postal[practice['code']] else practice_postal[practice['code']] 
    else:
        practice_postal[practice['code']] =practice['post_code']

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [8]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [9]:
joined = scripts[:]

for script in joined:
     if script['practice'] in practice_postal.keys():
        script['post_code'] =  practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [48]:
prescriptionByPostCode = {}
for element in joined:
    if element['post_code'] not in prescriptionByPostCode:
       prescriptionByPostCode[element['post_code']] = [element['items']]
    else:
        prescriptionByPostCode[element['post_code']].append(element['items'])

items_by_post = [sum(prescriptionByPostCode[x]) for x in prescriptionByPostCode]
keys = list(prescriptionByPostCode.keys())
pairs = (list(zip(keys, items_by_post))) 
items_by_post = [pair for pair in pairs if pair[1] > 0]
items_by_post.sort()

[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859), ('B70 7AW', 36531), ('BB11 2DL', 34356), ('BB2 1AX', 28254), ('BB3 1PY', 54514), ('BB4 5SL', 29388), ('BB7 2JG', 44585), ('BB8 0JZ', 54380), ('BB9 7SR', 38224), ('BD3 8QH', 21010), ('BH18 8EE', 39413), ('BH23 3AF', 32545), ('BL1 8TU', 26132), ('BL3 5HP', 27147), ('BL9 0NJ', 32062), ('BL9 0SN', 35275), ('CB9 8HF', 51337), ('CH1 4DS', 34915), ('CH65 6TG', 25090), ('CT11 8AD', 44358), ('CV1 4FS', 37210), ('CW1 3AW', 64104), ('CW5 5NX', 38797), ('CW7 1AT', 43164), ('DA1 2HA', 26075), ('DA11 8BZ', 24090), ('DN22 7XF', 43091), ('DN34 4GB', 48013), ('FY2 0JG', 69118), ('FY4 1TJ', 62886), ('FY5 2TZ', 44258), ('FY7 8GU', 34473), ('GL1 3PX', 38120), ('GL50 4DP', 74822), ('GU9 9QS', 32131), ('HA0 4UZ', 22755), ('HA3 7LT', 32113), ('HG1 5AR', 32684), ('HU7 4DW', 49107), ('KT14 6DH', 26758), ('KT6 6EZ', 38975), ('L31 0DJ', 32065), ('L36 7XY', 22965), ('L5 0QW', 24676), ('L7 6HD', 42569), ('LA1 1PN', 47335), ('LE10 1DS'

In [49]:
postal_totals = items_by_post[:100]

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [11]:
total_items_by_bnf_post = set()
for i in range(0, len(joined)):
    total_items_by_bnf_post.add((joined[i]["post_code"],joined[i]["bnf_name"],joined[i]["items"]))
total_items_by_bnf_post = list(total_items_by_bnf_post)
total_items_by_bnf_post.sort()

post_bnf_item = set()
for i in range(0, len(total_items_by_bnf_post)):
    post_bnf_item.add((total_items_by_bnf_post[i][0],total_items_by_bnf_post[i][1]))
post_bnf_item = list(post_bnf_item)
post_bnf_item.sort()
for i in range(0, len(post_bnf_item)):
    post_bnf_item[i] = list(post_bnf_item[i])
i = 0
j = 0
while j < len(post_bnf_item) and i < len(total_items_by_bnf_post):
    if post_bnf_item[j][0] == total_items_by_bnf_post[i][0] and post_bnf_item[j][1] == total_items_by_bnf_post[i][1]:
        post_bnf_item[j].append(total_items_by_bnf_post[i][2])
        i = i + 1
    else:
        j = j + 1

for i in range(0, len(post_bnf_item)):
    a = sum(post_bnf_item[i][2:])
    post_bnf_item[i] = post_bnf_item[i][:2]
    post_bnf_item[i].append(a)

total_items_by_bnf_post = post_bnf_item
    
post_codes = set()
for i in range(0, len(joined)):
    post_codes.add(joined[i]["post_code"])
post_codes = list(post_codes)
post_codes.sort()
    
post_dict = {}
for i in range(0, len(post_codes)):
    key_name = {post_codes[i] : []}
    post_dict.update(key_name)

bnf_add = ""
item_add = 0
for i in range(0, len(post_bnf_item)-1):
    if post_bnf_item[i][0] == post_bnf_item[i+1][0]:
        if item_add < post_bnf_item[i+1][2]:
            bnf_add = post_bnf_item[i+1][1]
            item_add = post_bnf_item[i+1][2]
    else:
        post_dict[post_bnf_item[i][0]] = [bnf_add, item_add]
        bnf_add = ""
        item_add = 0
total_items_by_post = post_dict

post_bnf_max_item = []

for i in range(0, 100):
    post_bnf_max_item.append((post_codes[i], post_dict[total_items_by_post[i]][0], total_items_by_post[post_codes[i]][1]/postal_totals[i][1]))

items_by_region = post_bnf_max_item
print(len(items_by_region))
assert len(total_items_by_bnf_post) == 141196

KeyError: 0

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

In [2]:
total_items_by_post = total_items_by_bnf_post.groupby(['post_code'],as_index=False).sum()
assert len(total_items_by_post) == 118

NameError: name 'total_items_by_bnf_post' is not defined

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [183]:
max_item_by_post = total_items_by_post.sort_values(by=['items','post_code'])
print(max_item_by_post)

    post_code   items
114  WS10 8SY   12598
78   SE15 5LJ   15258
104   W10 6DZ   16264
92   TN24 0GP   16955
64    NG7 3GW   17698
1     B18 7AL   19001
101  TW13 4GU   19584
0     B11 4BW   20673
12    BD3 8QH   21010
108  WD18 0JP   21034
67   NW10 8RY   21553
69   OL11 1DN   21567
85    SS0 7AF   21569
38    HA0 4UZ   22755
110   WN2 5NG   22795
45    L36 7XY   22965
113   WN7 2PE   23130
54    M11 4EJ   23166
28   DA11 8BZ   24090
89    ST1 4PB   24227
46     L5 0QW   24676
70    OL4 1YN   24687
3     B23 6DJ   24859
65    NG7 5HY   24903
81    SM3 8EP   24965
21   CH65 6TG   25090
58    ME8 8AA   25257
109  WF13 1HN   25456
73    RM3 9SU   25476
56    M30 0NU   25597
..        ...     ...
87    SS8 0JA   45848
82    SM6 0HY   46016
52    LN2 2JP   46173
48    LA1 1PN   47335
93   TN34 1BA   47440
95    TS1 2NX   47623
30   DN34 4GB   48013
53    LS9 9EF   48051
41    HU7 4DW   49107
49   LE10 1DS   49335
83    SR4 7XF   49843
61   NE24 1DX   50491
66   NN16 8DN   50771
19    CB9 

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [ ]:
#items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [196]:
grader.score.pw__items_by_region(items_by_region)

Your score:  0.9900000000000007


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*